# Learning from networks project: Evaluation of different Node Embedding algorithms
Members:<br>
- D'Emilio Filippo, id : 2120931
- Volpato Pietro, id : 2120825

## Embedding evaluation notebook

In [138]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
import gzip
import re
import os
import random
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE  # pip install imblearn
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.linear_model import LogisticRegression

# configuration
Here you can properly configure the names of the graphs and the names of the embedding strategies. Use meaningful names.

In [179]:
graph_keys = ["facebook","citation","biological","proteins", "CL"]
embedding_keys = ["LINE", "node2vec"]

### Functions temporary container

In [4]:
def plot_PCA(embeddings, graph_name = "G"):
    # Reduce dimensions to 2D using PCA
    pca = PCA(n_components=2)
    pca_embeddings = pca.fit_transform(embeddings)
    
    plt.scatter(pca_embeddings[:, 0], pca_embeddings[:, 1], s=10)
    plt.title(f"Visualization in 2D of the embeddings of {graph_name} graph.")
    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.figsize(5)
    plt.show()

#plot_PCA(embeddings_facebook["LINE"], "facebook")
#plot_PCA(embeddings_CL["LINE"], "CL")
#plot_PCA(embeddings_biological["LINE"], "biological")

# Loading the embeddings
Now we load the embeddings, which should be stored as a file in the /embeddings folder as a .npy file.<br>
*NOTE*: the file names must respect the format: "embeddings_{graph_key}_{embedding_key}.npy".<br>
Embeddings are stored in a dictionary of dictionaries.<br>
The first index refer to the graph (e.g. embeddings["facebook"] contains the embeddings of the facebook graph for every embedding technique).<br>
The second index refer to the embedding technique (e.g. embeddings["facebook"]["LINE"] cointans the embedding of facebook graph computed using LINE)

In [198]:
def load(name):
    """
    Loads a NumPy array from a file. If the file is not found, 
    displays a warning and returns None.

    name (str): The name of the file (without extension) to load from the 'embeddings' directory.
    
    return: np.ndarray or None: The loaded NumPy array, or None if the file is not found.
    """
    file_name = f"../result/{name}.npy"
    if not os.path.exists(file_name):
        print(f"Warning: File '{file_name}' not found.")
        return None

    emb = np.load(file_name)
    return emb

embeddings = {}
for k in graph_keys:
    embeddings[k] = {}

for graph_key in graph_keys:
    for emb_key in embedding_keys:
        emb_key.lower()
        graph_key.lower()
        s = f"embeddings_{graph_key}_{emb_key}"
        embeddings[graph_key][emb_key] = load(s)

# Loading the graphs
Selected graphs:
- Facebook_combined    https://snap.stanford.edu/data/ego-Facebook.html          
- cit-Helpth           https://networkrepository.com/cit-HepTh.php             
- bio-CE-CX            https://networkrepository.com/bio-CE-CX.php             
- proteins-full        https://networkrepository.com/PROTEINS-full.php ---- the graph has node labels
- COX2-MD              https://networkrepository.com/COX2-MD.php  ---- the graph has node labels

To run this notebook, adjust the paths to match where the files are saved in your PC.<br>
To keep paths as they are, download the repository.<br>
Graphs are stored as a dictionary: the key is the graph name, the value is the corresponding netowrkx graph.<br>

When it is created a networkX graph from a text file the node are renamed as integers form 0 to |V|-1, so that we can store the embeddings
on a matrix, and each row index corresponds to the embedding vector of the corrisponding node.

In [185]:
facebook_path = '../data/facebook/facebook_combined.txt.gz'
citation_path = '../data/citation/cit-HepTh.edges'
biological_path = '../data/biological/bio-CE-CX.edges'
proteins_path = "../data/proteins/PROTEINS-full.edges"
CL_path = "../data/CL-100K-1d8-L9.edges"

proteins_labels_path = "../data/proteins/PROTEINS-full.node_labels"
CL_labels_path = "../data/CL-100K-1d8-L9.node_labels"

In [186]:
def load_graph_edges(path):
    """
    For files with extension .edges
    nodes are renamed as integers, starting from 0
    """
    G = nx.Graph()
    with open(path, 'rt') as f:
        for line in f:
            if line.startswith('%'):  # Skip comment lines
                continue
            # Split the line based on spaces or commas
            data = re.split(r'[,\s]+', line.strip())
            if len(data) < 2:  # Skip lines that don't have at least two columns
                continue
            # Extract the first two columns (nodes)
            node1, node2 = int(data[0]), int(data[1])
            G.add_edge(node1, node2)
        
    return relabel_get_mapping(G)

def load_graph_with_gz(path):
    """
    For files with extension .txt.gz
    nodes are renamed as integers, starting from 0
    """
    G = nx.Graph()
    with gzip.open(path, 'rt') as f:
        for line in f:
            node1, node2 = map(int, line.strip().split())
            G.add_edge(node1, node2)
            
    return relabel_get_mapping(G)

def print_graphs_info(graphs):
    for k in graph_keys:
        G = graphs[k]
        print(f"{k}: |V|={len(G.nodes)}, |E|={len(G.edges)}")

def relabel_get_mapping(G):
    """
    Given a graph G, this function returns a graph where the nodes are relabeled as integers, form 0 to |V|-1.
    It is also returned the mapping from original name to relabeled name.
    """
    inverse_mapping = {i : node for i,node in enumerate(G.nodes)} # mappoing new_name : original_name
    direct_mapping = {node : i for i,node in enumerate(G.nodes)} # mapping original_name : new_name
    G = nx.relabel_nodes(G, direct_mapping)
    return G, direct_mapping

In [190]:
graphs = {}  # dictionary containg the graphs
mappings = {} # dictionary to contain the mappings. Original name : relabeled name
for k in graph_keys:
    mappings[k] = {}
    
# facebook graph is the only one .tar.gz        
graphs[graph_keys[0]], mappings[graph_keys[0]] = load_graph_with_gz(facebook_path)  # relabeling nodes to integer
graphs[graph_keys[1]], mappings[graph_keys[1]] = load_graph_edges(citation_path)
graphs[graph_keys[2]], mappings[graph_keys[2]] = load_graph_edges(biological_path)
graphs[graph_keys[3]], mappings[graph_keys[3]] = load_graph_edges(proteins_path)  # node labeled
graphs[graph_keys[4]], mappings[graph_keys[4]] = load_graph_edges(CL_path)  # node labeled

print_graphs_info(graphs)

facebook: |V|=4039, |E|=88234
citation: |V|=22908, |E|=2444798
biological: |V|=15229, |E|=245952
proteins: |V|=43471, |E|=81049
CL: |V|=92482, |E|=436611


# Reconstruction error
In order to avoid memory issues because of the allocation of very large adjacency matrices, it is computed the reconstruction error of the graph by comparing cosine similarity only for existing edges in the graph.

In [125]:
# The reconstruction error data structure is built as the embeddings data structure.
RE = {}
for k in graph_keys:
    RE[k] = {}

def reconstruction_error(G, embeddings):
    """
    Computes the reconstruction error of the graph by comparing cosine similarity
    only for existing edges in the graph, avoiding dense adjacency matrix computations.
    The reason is that for large graphs an exact computation causes memory issues, due to very large adjacency matrices.

    Parameters:
        G (networkx.Graph): The input graph.
        embeddings (NumPy array): numpy array containing the embeddings, each row is a node embedding

    Returns:
        float: The reconstruction error as the average squared difference for existing edges.
    """
    # Convert embeddings to matrix
    embedding_vectors = np.array([embeddings[node] for node in G.nodes])

    # Compute similarities only for existing edges
    error = 0
    for u, v in G.edges():
        u_vec = embedding_vectors[u].reshape(1, -1)
        v_vec = embedding_vectors[v].reshape(1, -1)
        sim = cosine_similarity(u_vec, v_vec)[0, 0]
        error += (1 - sim) ** 2

    return error / G.number_of_edges()

def print_reconstruction_error(err, graph_name , embedding_technique):
    print(f"RE of {graph_name} graph using {embedding_technique}: {err}")

def compute_all_reconstruction_errors(graph_keys, embedding_keys, show_results = True):
    for graph_key in graph_keys:
        if show_results:
            print(f"\nReconstruction errors for {graph_key} graph:\n")
        for emb_key in embedding_keys:     
            RE[graph_key][emb_key]= reconstruction_error(graphs[graph_key], embeddings[graph_key][emb_key])
            if show_results:
                print_reconstruction_error(RE[graph_key][emb_key], graph_key, emb_key)

## Compute the RE
Here you can compute the reconstruction error.<br>
- Set graph_keys_RE with the keys of the graphs you are interested. graph_keys_RE = graph_keys for all graphs.<br>
- set embedding_keys_RE with the keys of the embedding strategies you are interested. graph_keys_RE = embedding_keys for all embedding strategies.<br>

In [127]:
graph_keys_RE = ["proteins"]
embedding_keys_RE = ["LINE", "node2vec"]
compute_all_reconstruction_errors(graph_keys_RE, embedding_keys_RE, show_results = True)


Reconstruction errors for proteins graph:

RE of proteins graph using LINE: 0.08742006770010227
RE of proteins graph using node2vec: 0.419693647047861


# NODE CLASSIFICATION

## EXTRACT THE DATASET
The embeddings (features) are already well formed, and were loaded form the .npy files in the "result" folder.<br>
To extract the labels some considerations are needed. In the text files containing the labels each line has only one number (class label), and such element refers implicitly to the line number node, according to the original node names definition.<br>
Since when we load a graph we rename node as integers starting from 0, the mapping is applied to match each label with the correct node according
to the modified node names.<br>

NOTE: since isolated nodes are not present in the graph (leaded from .edges file), they are skippend, hence they label is not considered

In [216]:
def load_node_labels(file_path, graph_key):
    """
    Reads a file containing node labels and returns a dictionary mapping nodes to labels.
    The labels are assumed to be listed in sequential order: first label is relative to first node (according to original name), and so on...

    Parameters: file_path (str): Path to the node label file.
                graph_key : key of the graph, needed for the mappings between original and renamed nodes.
    

    Returns:
        node_labels (dict): A dictionary where keys are oroginal node names, values are their corresponding labels.
    """
    node_labels = {}
    with open(file_path, 'r') as file:
        i = 1 # keep track of incedes of original names because bad CL fata format
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace
            if not line:  # Skip empty lines
                continue

            # i is the node original name, but labels skip nodes that are isolated
            map = mappings[graph_key]
            while i not in map.keys():  # skip isolated nodes: they don't have label in CL data
                i += 1
            
            real_name = map[i]
            label = int(line)  # Parse the labels
            node_labels[real_name] = label  # Map the line number (node ID) to the label
            i += 1

    return node_labels

labels = {}
labels["proteins"] = load_node_labels(proteins_labels_path, "proteins")
labels["CL"] = load_node_labels(CL_labels_path, "CL")

## Analysis of the datasets
It is always a good idea to have a look at the datasets we are dealing with.<br>
- features are the embeddings of the nodes, which is an D-dimensional vector, where D is the dimension of the specific embeddings
we are using to training.
- the labels represent the classes of the nodes. We can analyze the labels set to see how many different classes there are and understand how balanced the dataset is.

In [217]:
def analyze_labels(labels):
    labels_count = {}
    for label in labels.values():
        if not label in labels_count.keys():
            labels_count[label] = 0
        labels_count[label] += 1
    print(f"Number of samples: {len(labels)}")
    print(f"Number of classes: {len(labels_count)}")
    for label in labels_count.keys():
        print(f"Samples of class {label}: {labels_count[label]}")

print("INFO DATASET PROTEINS GRAPH")
analyze_labels(labels["proteins"])

print("INFO DATASET CL GRAPH")
analyze_labels(labels["CL"])

INFO DATASET PROTEINS GRAPH
Number of samples: 43471
Number of classes: 3
Samples of class 0: 21151
Samples of class 1: 20931
Samples of class 2: 1389
INFO DATASET CL GRAPH
Number of samples: 92482
Number of classes: 9
Samples of class 8: 10276
Samples of class 9: 10274
Samples of class 2: 10276
Samples of class 6: 10276
Samples of class 1: 10276
Samples of class 3: 10276
Samples of class 5: 10276
Samples of class 4: 10276
Samples of class 7: 10276


## Train the model
A support vector machine is trained from the dataset provided in input.<br>
Class weights are used to handle the case of an imbalance dataset (some classes have considerably less examples than others).<br>
The model is trained on 80% of the dataset (training set), while 20% of the dataset (test set) is left to make predictions and evaluate the performances of the classifier.

In [203]:
def train_SVM(embeddings, labels):
    """
    Train and evaluate an SVM classifier for multi-class node classification.

    Parameters:
        embeddings (np.ndarray): Numpy array where each row is a node's embedding.
        labels (dict): Dictionary mapping node indices to their labels.

    Returns:
        dict: A dictionary with accuracy, F1 score, and a detailed classification report.
    """
    # Ensure X (features) and y (labels) are aligned
    X = np.array(embeddings)  # Node embeddings
    scaler = StandardScaler()
    X = scaler.fit_transform(X)  # normalization
    y = np.array([labels[i] for i in range(len(labels))])  # Ensure correct ordering of labels
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

    #smote = SMOTE(random_state=42) # (Synthetic Minority Oversampling Technique) 
    #X_train, y_train = smote.fit_resample(X_train, y_train)  #resampling to handle imbalance dataset

    clf = SVC(kernel='rbf', class_weight = 'balanced', decision_function_shape='ovo')  # 'ovo' = one-vs-one for multi-class
    #clf = BalancedRandomForestClassifier(n_estimators=100, random_state=42, sampling_strategy = 'all', bootstrap = False, replacement = True)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    # Evaluate the classifier
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='macro')
    report = classification_report(y_test, y_pred)

    # Print results
    print("SVM Classifier Results:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Macro F1-Score: {f1:.4f}")
    print("Classification Report:\n", report)

    # Return results as a dictionary
    return {"accuracy": accuracy, "macro_f1": f1, "report": report}

## Chose the graph and the embedding strategy to train the model

In [218]:
embedding_key = "LINE"
graph_key = "CL"
x = train_SVM(embeddings[graph_key][embedding_key], labels[graph_key])

SVM Classifier Results:
Accuracy: 0.1123
Macro F1-Score: 0.1123
Classification Report:
               precision    recall  f1-score   support

           1       0.10      0.11      0.11      2006
           2       0.12      0.12      0.12      2099
           3       0.11      0.11      0.11      1985
           4       0.10      0.11      0.10      2041
           5       0.12      0.13      0.13      2023
           6       0.11      0.10      0.11      2070
           7       0.12      0.11      0.11      2063
           8       0.12      0.11      0.11      2150
           9       0.11      0.10      0.11      2060

    accuracy                           0.11     18497
   macro avg       0.11      0.11      0.11     18497
weighted avg       0.11      0.11      0.11     18497



# LINK PREDICTION
The link prediction task consists on training a model, logistic regression in this case, that given as input the features of a pair of nodes predicts whether the edge between them exists in the graph or not.<br>

## PRODUCING THE DATASET
We decided to produce a perfectly balanced dataset in this way: all the positive examples are all the pair of nodes (actually the features we decided to use to represent them) for which exists an edge in the graph.<br>
The negative examples instead are created by chosing at random a pair of nodes, and if there is no edge between them such pair is a negative example. We repeat this procedure until the number of negative examples matches the number of positive examples<br>

We defined 3 possible choices for the features representing pair of nodes:
- The concatenation of the embeddings of the two nodes
- The concatenation of the embeddings + the cosine similarity between those (a scalar)
- Just the scalar cosine similarity between the two embeddings

In [172]:
def create_link_prediction_dataset(G, embeddings, add_cosine_sim = True):
    """
    Creates a dataset for link prediction. if add_cosine_sim = False, the features are the concatenation
    between the embedding of pair on nodes, if add_cosine_sim = True it is concatenated in addition the
    cosine similarity between the two embeddings (1 additional feature)

    Parameters:
        G (nx.Graph): The input graph.
        embeddings (np.ndarray): Node embeddings as a numpy array (indexed by node ID).
        add_cosine_sim: if True the cosine similarity is concatenated to the embeddings to produce the features

    Returns:
        X (np.ndarray): Feature pairs with cosine similarity as an additional feature.
        y (np.ndarray): Labels (1 for existing edges, 0 for non-existing edges).
    """
    positive_edges = list(G.edges())
    num_positive = len(positive_edges)

    # Generate negative edges
    nodes = list(G.nodes())
    negative_edges = set()
    while len(negative_edges) < num_positive:
        u, v = random.sample(nodes, 2)
        if not G.has_edge(u, v) and (u, v) not in negative_edges and (v, u) not in negative_edges:
            negative_edges.add((u, v))

    # Combine positive and negative edges
    all_edges = positive_edges + list(negative_edges)
    labels = [1] * len(positive_edges) + [0] * len(negative_edges)

    # Create features from embeddings
    X = []
    for u, v in all_edges:
        # Compute cosine similarity
        u_emb = embeddings[u].reshape(1, -1)
        v_emb = embeddings[v].reshape(1, -1)
        if add_cosine_sim:         # Concatenate embeddings and cosine similarity
            cosine_sim = cosine_similarity(u_emb, v_emb)[0, 0]
            feature = np.concatenate([embeddings[u], embeddings[v], [cosine_sim]])
        else:
            feature = np.concatenate([embeddings[u], embeddings[v]])
        X.append(feature)

    X = np.array(X)
    y = np.array(labels)

    return X, y

In [173]:
def create_link_prediction_dataset_only_cosine_similarity(G, embeddings):
    """
    Creates a dataset for link prediction, where each sample has a single feature (scalar), which is the cosine similarity
    between the two nodes adjacent to the edge.
    """
    positive_edges = list(G.edges())
    num_positive = len(positive_edges)

    # Generate negative edges
    nodes = list(G.nodes())
    negative_edges = set()
    while len(negative_edges) < num_positive:
        u, v = random.sample(nodes, 2)
        if not G.has_edge(u, v) and (u, v) not in negative_edges and (v, u) not in negative_edges:
            negative_edges.add((u, v))

    # Combine positive and negative edges
    all_edges = positive_edges + list(negative_edges)
    labels = [1] * len(positive_edges) + [0] * len(negative_edges)

    # Create features from embeddings
    cosine_similarities = []
    for u, v in all_edges:
        # Compute cosine similarity
        u_emb = embeddings[u].reshape(1, -1)
        v_emb = embeddings[v].reshape(1, -1)
        cosine_sim = cosine_similarity(u_emb, v_emb)[0, 0]
        cosine_similarities.append(cosine_sim)

    X = np.array(cosine_similarities).reshape(-1, 1)
    y = np.array(labels)

    return X, y

## TRAIN LOGISTIC REGRESSION CLASSIFIER

In [174]:
def LR_link_prediction(X,y):
    """
    Train a logistic regression classifier for link prediction
    """
    scaler = StandardScaler()
    X = scaler.fit_transform(X)  # normalization
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

    # Train logistic regression model
    clf = LogisticRegression(max_iter = 1000)
    clf.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = clf.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("ROC AUC Score:", roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))
    print("Classification Report:\n", classification_report(y_test, y_pred))

In [177]:
embedding_key = "node2vec"
graph_key = "proteins"
X,y = create_link_prediction_dataset_only_cosine_similarity(graphs[graph_key], embeddings[graph_key][embedding_key])
LR_link_prediction(X,y)

Accuracy: 0.5014188772362739
ROC AUC Score: 0.5044029397798646
Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.54      0.52     16178
           1       0.50      0.46      0.48     16242

    accuracy                           0.50     32420
   macro avg       0.50      0.50      0.50     32420
weighted avg       0.50      0.50      0.50     32420

